In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import files, drive
import os
# import cv2
import random
import pickle
from IPython.display import clear_output



In [ ]:
def load_dataset():
    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes.'.format(name=fn, length=len(uploaded[fn])))

    !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
    !kaggle datasets download -d jarvisgroot/brain-tumor-classification-mri-images
    !unzip brain-tumor-classification-mri-images.zip
    clear_output()

def load_dataset_drive():
    drive.mount('/content/drive')
    !mkdir -p ~/.kaggle/ && cp drive/MyDrive/Datasets/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
    !kaggle datasets download -d jarvisgroot/brain-tumor-classification-mri-images
    !unzip brain-tumor-classification-mri-images.zip
    clear_output()

# load_dataset()

In [ ]:
def setup_dataset():
    data_path = '/content/brain_tumor_mri/new_dataset/bt_images'

    training_data_path = os.path.join(data_path, '/content/brain_tumor_mri/new_dataset/training_data.pickle')
    # labels_path = os.path.join(data_path, '/content/brain_tumor_mri/new_dataset/labels.pickle')
    # test_data_path = os.path.join(data_path, '/content/test_images-20210704T210303Z-001/test_images')

    with open(training_data_path, 'rb') as training_data_file:
        training_data = pickle.load(training_data_file)

    # with open(labels_path, 'rb') as labels_file:
    #     labels = pickle.load(labels_file)

    return training_data

training_data = setup_dataset()
random.shuffle(training_data)

In [ ]:
# arr = np.array(training_data, dtype="object")
# X, Y = arr[:, 0], arr[:, 1]
# # X = np.array(X).reshape(-1, 512, 512, 3)
# # Y = np.array(Y)
# Y = Y - 1

In [ ]:
# X.shape

(3064, 512, 512, 3)

In [ ]:
X = []
y = []

for features, labels in training_data:
    X.append(features)
    y.append(labels)

X = np.array(X).reshape(-1, 512, 512, 3)
y = np.array(y)
y = y - 1

In [ ]:
del features
del labels
del training_data

In [ ]:
#94.14%
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), input_shape=X.shape[1:]),
    tf.keras.layers.Activation("softmax"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(128, (3,3)),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Activation("relu"),

    tf.keras.layers.Dense(3),
    tf.keras.layers.Activation('softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 510, 510, 64)      1792      
                                                                 
 activation_4 (Activation)   (None, 510, 510, 64)      0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 255, 255, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 253, 253, 128)     73856     
                                                                 
 activation_5 (Activation)   (None, 253, 253, 128)     0         
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 126, 126, 128)     0         
 g2D)                                                 

In [ ]:
model.compile(
    optimizer="adam",
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# X = tf.keras.applications.densenet.preprocess_input(X)
# temp = tf.convert_to_tensor(X)

model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1)

Epoch 1/10
87/87 [==============================] - 76s 656ms/step - loss: 3.0801 - accuracy: 0.7450 - val_loss: 0.3597 - val_accuracy: 0.8827
Epoch 2/10
87/87 [==============================] - 52s 600ms/step - loss: 0.1542 - accuracy: 0.9394 - val_loss: 0.1828 - val_accuracy: 0.9251
Epoch 3/10
87/87 [==============================] - 53s 608ms/step - loss: 0.0404 - accuracy: 0.9880 - val_loss: 0.2225 - val_accuracy: 0.9055
Epoch 4/10
87/87 [==============================] - 53s 608ms/step - loss: 0.0102 - accuracy: 0.9989 - val_loss: 0.1980 - val_accuracy: 0.9381
Epoch 5/10
87/87 [==============================] - 53s 609ms/step - loss: 0.0027 - accuracy: 0.9996 - val_loss: 0.2258 - val_accuracy: 0.9349
Epoch 6/10
87/87 [==============================] - 53s 609ms/step - loss: 7.3240e-04 - accuracy: 1.0000 - val_loss: 0.2380 - val_accuracy: 0.9381
Epoch 7/10
87/87 [==============================] - 53s 609ms/step - loss: 3.5771e-04 - accuracy: 1.0000 - val_loss: 0.2548 - val_accuracy

In [ ]:
# model.save('test_model.model')

# !zip -r test_model.zip test_model.model/
# !mv test_model.zip drive/MyDrive/Datasets/

In [ ]:
# file_size = os.path.getsize('drive/MyDrive/Datasets/test_model.zip')
# print("File Size is :", file_size, "bytes")

In [ ]:
data_path = '/content/brain_tumor_mri/new_dataset/bt_images'

training_data_path = os.path.join(data_path, '/content/brain_tumor_mri/new_dataset/training_data.pickle')
labels_path = os.path.join(data_path, '/content/brain_tumor_mri/new_dataset/labels.pickle')
test_data_path = os.path.join(data_path, '/content/test_images-20210704T210303Z-001/test_images')

In [ ]:
test_data = []

for img in os.listdir(test_data_path):
    img_array = cv2.imread(os.path.join(test_data_path, img))
    test_data.append([img_array])

In [ ]:
test_data = np.array(test_data).reshape(-1, 512, 512, 3)

In [ ]:
predictions = model.predict([test_data])
print(np.argmax(predictions[0]))

In [ ]:
plt.imshow(test_data[0])
plt.show()